In [1]:
import pandas as pd

# Login using e.g. `huggingface-cli login` to access this dataset
df = pd.read_parquet("../Nemotron_Personas.parquet")

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 22 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   uuid                        100000 non-null  object
 1   persona                     100000 non-null  object
 2   professional_persona        100000 non-null  object
 3   sports_persona              100000 non-null  object
 4   arts_persona                100000 non-null  object
 5   travel_persona              100000 non-null  object
 6   culinary_persona            100000 non-null  object
 7   skills_and_expertise        100000 non-null  object
 8   skills_and_expertise_list   100000 non-null  object
 9   hobbies_and_interests       100000 non-null  object
 10  hobbies_and_interests_list  100000 non-null  object
 11  career_goals_and_ambitions  100000 non-null  object
 12  sex                         100000 non-null  object
 13  age                         10

In [3]:
for column in df.columns:
    value=df.loc[1,column]
    print(f"Column: {column} \n Value: \n{value}\n")

Column: uuid 
 Value: 
3b5691bf-07cd-4e58-b85b-cff62faba2fd

Column: persona 
 Value: 
Quintin, a 40-year-old logistician from Converse, TX, balances curiosity with practicality, appreciating both new ideas and established methods, and maintains a unique blend of organization and flexibility in all aspects of life.

Column: professional_persona 
 Value: 
Quintin Pete Johnson, a logistician, combines his practical nature with a curiosity for new technologies, constantly seeking to optimize supply chains while remaining grounded in proven methods.

Column: sports_persona 
 Value: 
Quintin Pete Johnson, a dedicated fan of the San Antonio Spurs, maintains a balanced fitness routine, playing pick-up basketball on weekends and hiking at Government Canyon State Natural Area during weekdays.

Column: arts_persona 
 Value: 
They appreciate the gritty realism of Texas artist David Adickes' sculptures and the soulful melodies of Tejano musician Selena, often visiting the San Antonio Museum of Art

In [4]:
df.columns

Index(['uuid', 'persona', 'professional_persona', 'sports_persona',
       'arts_persona', 'travel_persona', 'culinary_persona',
       'skills_and_expertise', 'skills_and_expertise_list',
       'hobbies_and_interests', 'hobbies_and_interests_list',
       'career_goals_and_ambitions', 'sex', 'age', 'marital_status',
       'education_level', 'bachelors_field', 'occupation', 'city', 'state',
       'zipcode', 'country'],
      dtype='object')

In [17]:
not_required_columns=['zipcode', 'country']
persona_columns=['persona', 'professional_persona', 'sports_persona',
                'arts_persona', 'travel_persona', 'culinary_persona']
categorical_columns=['sex', 'marital_status','education_level', 
                     'bachelors_field', 'occupation', 'city', 'state']

In [18]:
df[persona_columns].head()

,persona,professional_persona,sports_persona,arts_persona,travel_persona,culinary_persona
0,"A disciplined, sociable visionary, Jonathan ba...","A retired manufacturing manager, Jonathan now ...","An avid golfer, Jonathan plays weekly at the W...","A history enthusiast, Jonathan often leads tou...","A seasoned, meticulous planner, Jonathan favor...","A fan of hearty, Midwestern comfort food, Jona..."
1,"Quintin, a 40-year-old logistician from Conver...","Quintin Pete Johnson, a logistician, combines ...","Quintin Pete Johnson, a dedicated fan of the S...",They appreciate the gritty realism of Texas ar...,"Quintin, a meticulous planner, balances family...",They delight in preparing complex Tex-Mex dish...
2,"Ashley, a passionate community advocate, balan...","Ashley, an aspiring union representative, exce...","Ashley, a dedicated Detroit Lions fan, maintai...","Ashley, a self-proclaimed 'Motown music enthus...","Ashley, a budget-conscious traveler, dreams of...","Ashley, a skilled home cook, loves preparing h..."
3,"Stephanie, always the first to volunteer, bala...","Stephanie, a customer service representative, ...","Stephanie, a die-hard Minnesota Vikings fan, p...","Stephanie, an avid reader and amateur painter,...","Stephanie, despite her love for the outdoors, ...","Stephanie, a self-taught cook, enjoys experime..."
4,"Sonia, a 70-year-old retiree, is a vibrant, im...","Sonia, a retired organizer with a creative sou...","Sonia, though not athletic, enjoys watching ba...","Sonia, a passionate artist, finds inspiration ...","Sonia, a seasoned traveler, plans meticulous i...","Sonia, an avid cook, delights in preparing com..."


In [7]:
df[categorical_columns].nunique()

sex                   2
marital_status        5
education_level       7
bachelors_field       5
occupation          566
city               9634
state                52
dtype: int64

## expertise_list

-TF-IDF 

In [8]:
import ast
df['hobbies_and_interests_list'] = df['hobbies_and_interests_list'].apply(ast.literal_eval)

In [9]:
total_hobbies = set()
for hobbies in df['hobbies_and_interests_list']:
    total_hobbies.update(hobbies)


len(total_hobbies)

42615

In [10]:
total_hobbies

{'trying new recipes inspired by puerto rican cuisine',
 'hunting (in season)',
 'attending community yard sales',
 'socializing with fellow farmers',
 'joining local history clubs',
 'dancing (line dancing, square dancing)',
 'volunteer work at community garden',
 'reading (crime novels, biographies)',
 'exploring new ideas and subjects',
 'cooking (traditional new england dishes)',
 'local arts and food scene exploration',
 'following the st. louis blues',
 'gardening, particularly rose cultivation',
 'hiking in watchung mountains',
 'attending intimate poetry readings and writing workshops',
 'spanish language exchange meetups',
 'reading (especially art and philosophy books)',
 'reading (bluegrass music, hip-hop, southern literature, contemporary fiction)',
 'playing lotería and other traditional mexican games',
 'reading medical thrillers',
 'reading art theory and philosophy books',
 'volunteer work (literacy programs)',
 'online learning courses',
 'sports (cardinals, diamondbac

In [19]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np

df_combined=pd.DataFrame()

# # Combine the text columns into a single 'full_persona' column
text_columns = persona_columns+['skills_and_expertise', 'skills_and_expertise_list', 'hobbies_and_interests', 'career_goals_and_ambitions']
df_combined['full_persona'] = df[text_columns].apply(lambda row: ' \n'.join(row.values.astype(str)), axis=1)


In [21]:
df_combined.head()

,full_persona
0,"A disciplined, sociable visionary, Jonathan ba..."
1,"Quintin, a 40-year-old logistician from Conver..."
2,"Ashley, a passionate community advocate, balan..."
3,"Stephanie, always the first to volunteer, bala..."
4,"Sonia, a 70-year-old retiree, is a vibrant, im..."


In [ ]:

# # Create a single vector for the combined text
model = SentenceTransformer('Qwen/Qwen3-Embedding-0.6B')
test_sentence = model.encode(df_combined['full_persona'].tolist())
# df_combined['persona_vector'] = df_combined['full_persona'].apply(lambda text: model.encode(text))


In [ ]:
import pickle



PICKLE_FILE = '../persona_embeddings(Qwen.Qwen3-Embedding-0.6B).pkl'

try:
    # 1. Try to access the variable.
    # We use '_' to just check existence without doing anything.
    _ = test_sentence
    print("Variable 'test_sentence' already exists in memory.")

except NameError:
    # 2. If it raises a NameError, it doesn't exist.
    print("Variable not in memory. Trying to load from pickle...")
    
    try:
        # 3. Try to load it from the file.
        with open(PICKLE_FILE, 'rb') as f:
            test_sentence = pickle.load(f)
        print("Successfully loaded 'test_sentence' from file.")

    except FileNotFoundError:
        # 4. Handle the case where the file doesn't exist.
        with open(PICKLE_FILE, 'wb') as f:
            pickle.dump(test_sentence, f)
        print(f"Pickle file '{PICKLE_FILE}' has been created with 'test_sentence'.")
        
    except Exception as e:
        # 5. Handle other potential pickle errors (corrupt file, etc.)
        print(f"An error occurred while loading the pickle file: {e}")

# Now you can safely use the variable (if it was loaded or existed)
# You might want another check here if loading could fail
if 'test_sentence' in locals():
    print(f"\nFinal variable content: {test_sentence}")
else:
    print("\nVariable could not be loaded or created.")

In [ ]:
import json

JSON_FILE = '../all_hobbies.json'

try:
    with open(JSON_FILE, 'r') as f:
        total_hobbies = json.load(f)
    print("Successfully loaded 'total_hobbies' from JSON file.")

except FileNotFoundError:
    print(f"JSON file '{JSON_FILE}' not found.")
    with open(JSON_FILE, 'w') as f:
        json.dump(list(total_hobbies), f)

except json.JSONDecodeError:
    print(f"Error decoding JSON from file '{JSON_FILE}'.")
